In [50]:
# Import dependencies
import pandas as pd
import chardet
# For data base creation
from sqlalchemy import create_engine
# from config import db_password
import os
import os.path, sys
import glob
import re

In [51]:
# tournament file location
file = '../../atp-world-tour-tennis-data/csv/1_tournaments/'

In [5]:
# def for cleaning tournament data
def clean_tournament(file):
    # assign path
    path, dirs, files = next(os.walk(file))
    file_count = len(files)
    # create empty list
    dataframes_list = []
    
    # append datasets to the list
    for i in range(file_count):
        with open(file+files[i], 'rb') as rawdata:
            result = chardet.detect(rawdata.read(100000))
        temp_df = pd.read_csv(file+files[i], encoding=result['encoding'], header=None)
        dataframes_list.append(temp_df)

    df = pd.concat(dataframes_list)
    df.drop(columns=[1,4,5,8,9,10,12,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30], inplace=True)
    df = df.reindex(columns=[0,2,3,7,6,11,13,14,22])
    header_df = pd.read_csv("../temp/Tourney_header.csv")
    df.columns = header_df.columns
    #Combinding coloumns 2,3
    df["Name"] = df["Name"]+"-"+ df["temp"]
    df = df.drop(columns =['temp'])
    df = df.reset_index()
    return df

In [182]:
# running CT function
df = clean_tournament(file)
df.head(22)

,index,Turney_id,Name,Date,Location,Draw_size,Conditions,Surface,Winner_id
0,0,1877-540,Grand Slam-Wimbledon,1877.07.09,"London, Great Britain",32,Outdoor,Grass,gi91
1,1,1878-540,Grand Slam-Wimbledon,1878.07.08,"London, Great Britain",64,Outdoor,Grass,hg50
2,2,1879-540,Grand Slam-Wimbledon,1879.07.07,"London, Great Britain",64,Outdoor,Grass,hg35
3,3,1880-540,Grand Slam-Wimbledon,1880.07.05,"London, Great Britain",64,Outdoor,Grass,hg35
4,4,1881-540,Grand Slam-Wimbledon,1881.07.02,"London, Great Britain",64,Outdoor,Grass,rg71
...,...,...,...,...,...,...,...,...,...
17,17,1887-560,Grand Slam-US Championships,1887.08.22,"Newport, United States",17,Outdoor,Grass,sw16
18,18,1888-540,Grand Slam-Wimbledon,1888.07.10,"London, Great Britain",32,Outdoor,Grass,rg47
19,19,1888-560,Grand Slam-US Championships,1888.08.20,"Newport, United States",33,Outdoor,Grass,sw16
20,20,1889-540,Grand Slam-Wimbledon,1889.07.01,"London, Great Britain",32,Outdoor,Grass,rg71


In [377]:
def clean_Match_Score(file):
    # finds information and puts it in a df
    file = '../../atp-world-tour-tennis-data/csv/2_match_scores/'

    # assign path
    path, dirs, files = next(os.walk(file))
    file_count = len(files)
    # create empty list
    dataframes_list = []
    
    # append datasets to the list
    for i in range(file_count):
        temp_df = pd.read_csv(file+files[i], encoding='utf-8', header=None)
        dataframes_list.append(temp_df)

    df_W = pd.concat(dataframes_list)
    df_L = pd.concat(dataframes_list)
    # dropping un needed winner stats
    df_W.drop(columns=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,21,22,24,25,26,28,29,30,31,32,33], inplace=True)
    # Dropping un needed Loser stats
    df_L.drop(columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,21,22,24,25,26,28,29,30,31,32,33,35], inplace=True)
    # reorder columns to match headers
    df_W =df_W.reindex(columns = [34,0,20,16,27,5,23])
    df_L=df_L.reindex(columns=[34,0,23,16,27,5,20])
    # setting winner&Loser headers
    df_W_Header = pd.read_csv("../temp/Match_score_header_W.csv")
    df_L_Header = pd.read_csv("../temp/Match_score_header_L.csv")
    # connecting winner&loser headers
    df_W[34] = pd.Series(df_W[0].astype(str) + '-' + df_W[20].astype(str) + '-' + df_W[23].astype(str), index=df_W.index)
    df_W.drop(columns=[23], inplace=True)
    df_L.drop(columns= [20], inplace=True)
    df_W.columns = df_W_Header.columns
    df_L.columns = df_L_Header.columns
    # Setting player_results
    df_W['Player_result'] = pd.Series('Won', index=df_W.index)
    df_L['Player_result'] = pd.Series('Lost', index=df_L.index)
    # Combining winner and loser dataframes
    df = pd.concat([df_W, df_L])
    df = df.reset_index()
    replace_values = {'61': '6-1', '62': '6-2', '63': '6-3', '64': '6-4', '65': '6-5', '16': '1-6', '26': '2-6', '36': '3-6', '46': '4-6', '56': '5-6', '76':'7-6', '67': '6-7', \
                      '57': '5-7', '75': '7-5', '68': '6-8', '86': '8-6', '79': '7-9', '97': '9-7', '810': '8-10', '108': '10-8', '911': '9-11', '119': '11-9', '1012': '10-12', \
                      '1210': '12-10', '1113': '11-13', '1311': '13-11', '1214': '12-14', '1412': '14-12', '1315': '13-15', '1513': '15-13', '1416': '14-16', '1614': '16-14', \
                      '1517': '15-17', '1715': '17-15', '1618': '16-18', '1816': '18-16', '1719': '17-19', '1917': '19-17', '1820': '18-20', '2018': '20-18', '1921': '19-21', \
                      '2119': '21-19', '2022': '20-22', '2220': '22-20', '2123': '21-23', '2321': '23-21', '2224': '22-24', '2422': '24-22', '2325': '23-25', '2523': '25-23', \
                      '2426': '24-26', '2624': '26-24', '7068': '70-68', '06': '0-6', '60': '6-0'}
    for key, value in replace_values.items():
        df['Score'] = df['Score'].str.replace(key, value)
    return df

In [378]:
df = clean_Match_Score(file)
